# StockSage.AI - Data Exploration & API Testing

This notebook tests all our data sources and API connections to ensure everything is working properly.

## 🎯 What we'll test:
1. **Market Data** - yfinance (no API key needed)
2. **Economic Data** - FRED API 
3. **News Data** - NewsAPI
4. **AI/ML** - OpenAI API & FinBERT
5. **Visualization** - Plotly charts


In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import yfinance as yf
import requests
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()
print("✅ Libraries imported successfully!")
print(f"📅 Current date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Check if .env file exists
if os.path.exists('.env'):
    print("✅ .env file found")
else:
    print("⚠️  .env file not found - you'll need API keys for full testing")


✅ Libraries imported successfully!
📅 Current date: 2025-07-09 19:33:46
⚠️  .env file not found - you'll need API keys for full testing


## 📊 1. Market Data Testing (yfinance)

Testing stock market data collection - **no API key required!**


In [3]:
# Test yfinance - No API key needed!
print("🔍 Testing Market Data Collection...\n")

try:
    # Get Apple stock data
    ticker = "AAPL"
    stock = yf.Ticker(ticker)
    
    # Get historical data
    hist_data = stock.history(period="1mo")
    print(f"✅ Successfully downloaded {ticker} data:")
    print(f"   - Date range: {hist_data.index[0].date()} to {hist_data.index[-1].date()}")
    print(f"   - Records: {len(hist_data)}")
    print(f"   - Latest close price: ${hist_data['Close'].iloc[-1]:.2f}")
    
    # Get company info
    info = stock.info
    print(f"   - Company: {info.get('longName', 'N/A')}")
    print(f"   - Sector: {info.get('sector', 'N/A')}")
    
    # Display last 5 days
    print("\n📈 Last 5 trading days:")
    display(hist_data.tail().round(2))
    
except Exception as e:
    print(f"❌ Error with yfinance: {e}")
    print("💡 Make sure you have internet connection")


🔍 Testing Market Data Collection...

✅ Successfully downloaded AAPL data:
   - Date range: 2025-06-10 to 2025-07-09
   - Records: 20
   - Latest close price: $211.14
   - Company: Apple Inc.
   - Sector: Technology

📈 Last 5 trading days:


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-07-02 00:00:00-04:00,208.91,213.34,208.14,212.44,67941800,0.0,0.0
2025-07-03 00:00:00-04:00,212.15,214.65,211.81,213.55,34955800,0.0,0.0
2025-07-07 00:00:00-04:00,212.68,216.23,208.80,209.95,50229000,0.0,0.0
2025-07-08 00:00:00-04:00,210.10,211.43,208.45,210.01,42848900,0.0,0.0
2025-07-09 00:00:00-04:00,209.53,211.33,207.22,211.14,48676300,0.0,0.0


## 🏛️ 2. Economic Data Testing (FRED API)

Testing Federal Reserve economic data - **requires FREE API key**


In [4]:
# Test FRED API
print("🔍 Testing Economic Data Collection...\n")

FRED_API_KEY = os.getenv('FRED_API_KEY')

if not FRED_API_KEY:
    print("⚠️  FRED API key not set. Please:")
    print("   1. Register at: https://fred.stlouisfed.org/docs/api/")
    print("   2. Add key to .env file: FRED_API_KEY=your_actual_key")
else:
    try:
        def get_fred_data(series_id, limit=3):
            url = "https://api.stlouisfed.org/fred/series/observations"
            params = {
                'series_id': series_id,
                'api_key': FRED_API_KEY,
                'file_type': 'json',
                'limit': limit,
                'sort_order': 'desc'
            }
            response = requests.get(url, params=params)
            
            if response.status_code == 200:
                data = response.json()
                return pd.DataFrame(data['observations'])
            else:
                print(f"❌ FRED API Error: {response.status_code}")
                return None
        
        # Test key economic indicators
        indicators = {
            'Federal Funds Rate': 'FEDFUNDS',
            'Unemployment Rate': 'UNRATE',
            'Consumer Price Index': 'CPIAUCSL'
        }
        
        print("✅ FRED API connection successful!\n")
        
        for name, series_id in indicators.items():
            data = get_fred_data(series_id)
            if data is not None and not data.empty:
                latest = data.iloc[0]
                print(f"📊 {name} ({series_id}):")
                print(f"   Latest: {latest['value']} ({latest['date']})")
            else:
                print(f"❌ Failed to get {name} data")
            print()
            
    except Exception as e:
        print(f"❌ Error with FRED API: {e}")


🔍 Testing Economic Data Collection...

✅ FRED API connection successful!

📊 Federal Funds Rate (FEDFUNDS):
   Latest: 4.33 (2025-06-01)

📊 Unemployment Rate (UNRATE):
   Latest: 4.1 (2025-06-01)

📊 Consumer Price Index (CPIAUCSL):
   Latest: 320.580 (2025-05-01)



## ✅ 3. Quick FinBERT Sentiment Test

Testing sentiment analysis without API keys


In [ ]:
# Test FinBERT for sentiment analysis
print("🔍 Testing FinBERT Sentiment Analysis...\n")

try:
    from transformers import pipeline
    
    # Load FinBERT sentiment pipeline (this downloads the model on first use)
    print("📥 Loading FinBERT model (may take a moment on first run)...")
    sentiment_pipeline = pipeline(
        "sentiment-analysis", 
        model="ProsusAI/finbert",
        return_all_scores=True
    )
    
    # Test sentences
    test_sentences = [
        "Apple reported strong quarterly earnings beating expectations.",
        "The stock market crashed due to inflation concerns.",
        "Tesla's production numbers look very promising this quarter."
    ]
    
    print("✅ FinBERT loaded successfully!\n")
    print("🧪 Testing sentiment analysis:")
    
    for i, sentence in enumerate(test_sentences, 1):
        result = sentiment_pipeline(sentence)
        best_sentiment = max(result[0], key=lambda x: x['score'])
        
        print(f"\n📝 Test {i}: {sentence}")
        print(f"   Sentiment: {best_sentiment['label']} (confidence: {best_sentiment['score']:.3f})")
        
except Exception as e:
    print(f"❌ Error with FinBERT: {e}")
    print("💡 Note: FinBERT requires internet connection for first-time download")
    print("   This is normal and the model will be cached for future use.")


🔍 Testing FinBERT Sentiment Analysis...

📥 Loading FinBERT model (may take a moment on first run)...


Device set to use cpu


✅ FinBERT loaded successfully!

🧪 Testing sentiment analysis:

📝 Test 1: Apple reported strong quarterly earnings beating expectations.
   Sentiment: positive (confidence: 0.954)

📝 Test 2: The stock market crashed due to inflation concerns.
   Sentiment: negative (confidence: 0.948)

📝 Test 3: Tesla's production numbers look very promising this quarter.
   Sentiment: positive (confidence: 0.948)


# Alpha Vantage Testing

In [12]:
# 💡 Alpha Vantage Alternative - Using Different Function
print("🔄 Trying Alpha Vantage with alternative endpoint...\n")

ALPHA_VANTAGE_KEY = os.getenv('ALPHA_VANTAGE_API_KEY')

if ALPHA_VANTAGE_KEY and ALPHA_VANTAGE_KEY != 'your_alpha_vantage_key_here':
    try:
        # Try a simpler API call that uses fewer quota
        url = "https://www.alphavantage.co/query"
        params = {
            'function': 'GLOBAL_QUOTE',  # Simpler endpoint, uses less quota
            'symbol': 'AAPL',
            'apikey': ALPHA_VANTAGE_KEY
        }
        
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            
            print(f"📊 Response Keys: {list(data.keys())}")
            
            if 'Global Quote' in data:
                quote = data['Global Quote']
                print("✅ Alpha Vantage working with Global Quote!")
                print(f"   Symbol: {quote.get('01. symbol', 'N/A')}")
                print(f"   Price: ${float(quote.get('05. price', 0)):.2f}")
                print(f"   Change: {quote.get('09. change', 'N/A')} ({quote.get('10. change percent', 'N/A')})")
                print(f"   Volume: {quote.get('06. volume', 'N/A')}")
                
            elif 'Information' in data:
                print("ℹ️ Still hitting rate limit with simpler endpoint")
                print("   💡 Your account has used up today's quota")
                print("   ⏰ Reset time: Usually midnight EST")
                
        print("\n🔧 Rate Limit Management Tips:")
        print("   1. Alpha Vantage tracks by API key, not IP")
        print("   2. Free tier: 25 calls/day, 5 calls/minute")
        print("   3. Cache responses locally to avoid repeat calls")
        print("   4. Use yfinance as primary source, Alpha Vantage as backup")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        
print("\n✅ RECOMMENDATION:")
print("   Since yfinance is working perfectly, use it as your primary data source.")
print("   Alpha Vantage can be a backup or for special indicators not in yfinance.")


🔄 Trying Alpha Vantage with alternative endpoint...

📊 Response Keys: ['Global Quote']
✅ Alpha Vantage working with Global Quote!
   Symbol: AAPL
   Price: $211.14
   Change: 1.1300 (0.5381%)
   Volume: 48605601

🔧 Rate Limit Management Tips:
   1. Alpha Vantage tracks by API key, not IP
   2. Free tier: 25 calls/day, 5 calls/minute
   3. Cache responses locally to avoid repeat calls
   4. Use yfinance as primary source, Alpha Vantage as backup

✅ RECOMMENDATION:
   Since yfinance is working perfectly, use it as your primary data source.
   Alpha Vantage can be a backup or for special indicators not in yfinance.


In [ ]:
# Test Finnhub API
print("🔍 Testing Finnhub API...\n")

FINNHUB_API_KEY = os.getenv('FINNHUB_API_KEY')

if not FINNHUB_API_KEY or FINNHUB_API_KEY == 'your_finnhub_key_here':
    print("⚠️  Finnhub API key not set. Please:")
    print("   1. Register at: https://finnhub.io/register")
    print("   2. Add key to .env file: FINNHUB_API_KEY=your_actual_key")
    print("   💡 Free tier: 60 calls/minute")
else:
    try:
        base_url = "https://finnhub.io/api/v1"
        headers = {'X-Finnhub-Token': FINNHUB_API_KEY}
        
        # Test 1: Get stock quote
        print("📊 Testing stock quote...")
        quote_url = f"{base_url}/quote"
        params = {'symbol': 'AAPL'}
        
        response = requests.get(quote_url, params=params, headers=headers)
        
        if response.status_code == 200:
            quote_data = response.json()
            
            if 'c' in quote_data and quote_data['c'] != 0:
                print("✅ Finnhub stock quote successful!")
                print(f"   📈 AAPL Current Price: ${quote_data['c']:.2f}")
                print(f"   📊 Change: ${quote_data['d']:.2f} ({quote_data['dp']:.2f}%)")
                print(f"   📅 High: ${quote_data['h']:.2f} | Low: ${quote_data['l']:.2f}")
                
                # Test 2: Get company news
                print("\\n📰 Testing company news...")
                news_url = f"{base_url}/company-news"
                
                # Get news from last 7 days
                from_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
                to_date = datetime.now().strftime('%Y-%m-%d')
                
                news_params = {
                    'symbol': 'AAPL',
                    'from': from_date,
                    'to': to_date
                }
                
                news_response = requests.get(news_url, params=news_params, headers=headers)
                
                if news_response.status_code == 200:
                    news_data = news_response.json()
                    
                    if news_data and len(news_data) > 0:
                        print(f"✅ Found {len(news_data)} news articles for AAPL")
                        
                        # Show first 2 articles
                        for i, article in enumerate(news_data[:2], 1):
                            headline = article.get('headline', 'No headline')[:50]
                            source = article.get('source', 'Unknown')
                            
                            print(f"   📰 Article {i}: {headline}...")
                            print(f"      Source: {source}")
                    else:
                        print("⚠️ No recent news found for AAPL")
                else:
                    print(f"❌ News request failed: {news_response.status_code}")
                    
            else:
                print("❌ Invalid quote data received")
                
        else:
            print(f"❌ Finnhub API Error: {response.status_code}")
            if response.status_code == 429:
                print("   💡 Rate limit exceeded - free tier allows 60 calls/minute")
            elif response.status_code == 401:
                print("   💡 Unauthorized - check your API key")
                
    except Exception as e:
        print(f"❌ Error with Finnhub API: {e}")


In [ ]:
# 🎯 UPDATED FINAL TESTING SUMMARY
print("="*60)
print("🎯 STOCKSAGE.AI - COMPLETE SYSTEM STATUS CHECK")
print("="*60)

# Test what's working without API keys
working_systems = []
needs_api_keys = []

# Test basic imports
try:
    import yfinance as yf
    import pandas as pd
    import plotly.graph_objects as go
    working_systems.append("📊 Core Libraries (pandas, plotly)")
    working_systems.append("📈 Market Data (yfinance)")
except:
    needs_api_keys.append("❌ Core Libraries")

# Check all API keys
fred_key = os.getenv('FRED_API_KEY')
news_key = os.getenv('NEWS_API_KEY')
openai_key = os.getenv('OPENAI_API_KEY')
alpha_vantage_key = os.getenv('ALPHA_VANTAGE_API_KEY')
finnhub_key = os.getenv('FINNHUB_API_KEY')

# Essential APIs
if fred_key and fred_key != 'your_fred_api_key_here':
    working_systems.append("🏛️ Economic Data (FRED)")
else:
    needs_api_keys.append("⚠️ FRED API (Economic Data)")

if news_key and news_key != 'your_news_api_key_here':
    working_systems.append("📰 News Data (NewsAPI)")
else:
    needs_api_keys.append("⚠️ News API")

if openai_key and openai_key != 'your_openai_api_key_here':
    working_systems.append("🤖 OpenAI API")
else:
    needs_api_keys.append("⚠️ OpenAI API")

# Optional Enhanced APIs
if alpha_vantage_key and alpha_vantage_key != 'your_alpha_vantage_key_here':
    working_systems.append("📈 Alpha Vantage (Alternative Market Data)")
else:
    needs_api_keys.append("⚠️ Alpha Vantage API (Optional)")

if finnhub_key and finnhub_key != 'your_finnhub_key_here':
    working_systems.append("📰 Finnhub (Financial News & Data)")
else:
    needs_api_keys.append("⚠️ Finnhub API (Optional)")

# Print summary
print("\n✅ WORKING SYSTEMS:")
for system in working_systems:
    print(f"  {system}")

print("\n⚠️  NEEDS SETUP:")
for system in needs_api_keys:
    print(f"  {system}")

# Calculate status
total_systems = len(working_systems) + len(needs_api_keys)
essential_working = len([s for s in working_systems if not s.endswith("(Alternative Market Data)") and not s.endswith("(Financial News & Data)")])
essential_total = total_systems - 2  # Subtract optional APIs

print(f"\n🚀 STATUS: {len(working_systems)}/{total_systems} total systems ready")
print(f"   Essential: {essential_working}/{essential_total} core systems ready")

if essential_working >= 3:  # Core libraries + market data + one data source
    print("🎉 CORE SYSTEMS READY! You can start building StockSage.AI!")
    if len(needs_api_keys) > 0:
        print("💡 Add optional APIs to unlock full capabilities")
else:
    print("\n📝 TO DO:")
    print("  1. Get API keys from the URLs shown below")
    print("  2. Add them to your .env file")
    print("  3. Rerun this notebook")
    
print("\n🔗 API Registration Links:")
print("  🏛️ FRED (Economic): https://fred.stlouisfed.org/docs/api/")
print("  📰 NewsAPI: https://newsapi.org/")
print("  🤖 OpenAI: https://platform.openai.com/api-keys")
print("  📈 Alpha Vantage: https://www.alphavantage.co/support/#api-key")
print("  📰 Finnhub: https://finnhub.io/register")

print(f"\n✨ NEXT STEP: Create data collection scripts in src/data/!")
print("   🎯 You now have a complete testing framework for all APIs")


## 📰 5. Finnhub API Testing

Testing financial news and market data - **requires FREE API key**


In [9]:
# Test Finnhub API
print("🔍 Testing Finnhub API...\n")

FINNHUB_API_KEY = os.getenv('FINNHUB_API_KEY')

if not FINNHUB_API_KEY or FINNHUB_API_KEY == 'your_finnhub_key_here':
    print("⚠️  Finnhub API key not set. Please:")
    print("   1. Register at: https://finnhub.io/register")
    print("   2. Add key to .env file: FINNHUB_API_KEY=your_actual_key")
    print("   💡 Free tier: 60 calls/minute")
else:
    try:
        base_url = "https://finnhub.io/api/v1"
        headers = {'X-Finnhub-Token': FINNHUB_API_KEY}
        
        # Test 1: Get stock quote
        print("📊 Testing stock quote...")
        quote_url = f"{base_url}/quote"
        params = {'symbol': 'AAPL'}
        
        response = requests.get(quote_url, params=params, headers=headers)
        
        if response.status_code == 200:
            quote_data = response.json()
            
            if 'c' in quote_data and quote_data['c'] != 0:
                print("✅ Finnhub stock quote successful!")
                print(f"   📈 AAPL Current Price: ${quote_data['c']:.2f}")
                print(f"   📊 Change: ${quote_data['d']:.2f} ({quote_data['dp']:.2f}%)")
                print(f"   📅 High: ${quote_data['h']:.2f} | Low: ${quote_data['l']:.2f}")
                print(f"   📅 Previous Close: ${quote_data['pc']:.2f}")
                
                # Test 2: Get company news
                print("\\n📰 Testing company news...")
                news_url = f"{base_url}/company-news"
                
                # Get news from last 7 days
                from_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
                to_date = datetime.now().strftime('%Y-%m-%d')
                
                news_params = {
                    'symbol': 'AAPL',
                    'from': from_date,
                    'to': to_date
                }
                
                news_response = requests.get(news_url, params=news_params, headers=headers)
                
                if news_response.status_code == 200:
                    news_data = news_response.json()
                    
                    if news_data and len(news_data) > 0:
                        print(f"✅ Found {len(news_data)} news articles for AAPL")
                        
                        # Show first 3 articles
                        for i, article in enumerate(news_data[:3], 1):
                            headline = article.get('headline', 'No headline')[:60]
                            source = article.get('source', 'Unknown')
                            datetime_str = datetime.fromtimestamp(article.get('datetime', 0)).strftime('%Y-%m-%d %H:%M')
                            
                            print(f"   📰 Article {i}: {headline}...")
                            print(f"      Source: {source} | Date: {datetime_str}")
                    else:
                        print("⚠️ No recent news found for AAPL")
                else:
                    print(f"❌ News request failed: {news_response.status_code}")
                
                # Test 3: Get basic financials
                print("\\n💰 Testing basic financials...")
                financials_url = f"{base_url}/stock/metric"
                financials_params = {
                    'symbol': 'AAPL',
                    'metric': 'all'
                }
                
                financials_response = requests.get(financials_url, params=financials_params, headers=headers)
                
                if financials_response.status_code == 200:
                    financials_data = financials_response.json()
                    
                    if 'metric' in financials_data:
                        metrics = financials_data['metric']
                        print("✅ Basic financials retrieved:")
                        
                        # Show key metrics if available
                        if 'peNormalizedAnnual' in metrics:
                            print(f"   📊 P/E Ratio: {metrics['peNormalizedAnnual']:.2f}")
                        if 'marketCapitalization' in metrics:
                            market_cap = metrics['marketCapitalization']
                            print(f"   💰 Market Cap: ${market_cap:,.0f}M")
                        if 'beta' in metrics:
                            print(f"   📈 Beta: {metrics['beta']:.2f}")
                    else:
                        print("⚠️ Financial metrics not available")
                else:
                    print(f"❌ Financials request failed: {financials_response.status_code}")
                    
            else:
                print("❌ Invalid quote data received")
                print(f"   Response: {quote_data}")
                
        else:
            print(f"❌ Finnhub API Error: {response.status_code}")
            if response.status_code == 429:
                print("   💡 Rate limit exceeded - free tier allows 60 calls/minute")
            elif response.status_code == 401:
                print("   💡 Unauthorized - check your API key")
                
    except Exception as e:
        print(f"❌ Error with Finnhub API: {e}")


🔍 Testing Finnhub API...

📊 Testing stock quote...
✅ Finnhub stock quote successful!
   📈 AAPL Current Price: $211.14
   📊 Change: $1.13 (0.54%)
   📅 High: $211.33 | Low: $207.22
   📅 Previous Close: $210.01
\n📰 Testing company news...
✅ Found 234 news articles for AAPL
   📰 Article 1: Meta Poached Apple’s Pang With Pay Package Over $200 Million...
      Source: Yahoo | Date: 2025-07-09 16:31
   📰 Article 2: Audiio Hires CD Baby Sync Industry Veteran Brett Byrd To Lea...
      Source: Yahoo | Date: 2025-07-09 16:11
   📰 Article 3: Are Analysts Getting Too Bullish on Microsoft and Nvidia?...
      Source: Yahoo | Date: 2025-07-09 16:05
\n💰 Testing basic financials...
✅ Basic financials retrieved:
   📊 P/E Ratio: 33.29
   💰 Market Cap: $3,120,766M
   📈 Beta: 1.11


In [7]:
# 🎯 FINAL TESTING SUMMARY
print("="*50)
print("🎯 STOCKSAGE.AI - SYSTEM STATUS CHECK")
print("="*50)

# Test what's working without API keys
working_systems = []
needs_api_keys = []

# Test basic imports
try:
    import yfinance as yf
    import pandas as pd
    import plotly.graph_objects as go
    working_systems.append("📊 Core Libraries (pandas, plotly)")
    working_systems.append("📈 Market Data (yfinance)")
except:
    needs_api_keys.append("❌ Core Libraries")

# Check API keys
fred_key = os.getenv('FRED_API_KEY')
news_key = os.getenv('NEWS_API_KEY')
openai_key = os.getenv('OPENAI_API_KEY')

if fred_key and fred_key != 'your_fred_api_key_here':
    working_systems.append("🏛️ Economic Data (FRED)")
else:
    needs_api_keys.append("⚠️ FRED API (Economic Data)")

if news_key and news_key != 'your_news_api_key_here':
    working_systems.append("📰 News Data (NewsAPI)")
else:
    needs_api_keys.append("⚠️ News API")

if openai_key and openai_key != 'your_openai_api_key_here':
    working_systems.append("🤖 OpenAI API")
else:
    needs_api_keys.append("⚠️ OpenAI API")

# Print summary
print("\n✅ WORKING SYSTEMS:")
for system in working_systems:
    print(f"  {system}")

print("\n⚠️  NEEDS SETUP:")
for system in needs_api_keys:
    print(f"  {system}")

# Next steps
print(f"\n🚀 STATUS: {len(working_systems)}/{len(working_systems) + len(needs_api_keys)} systems ready")

if len(needs_api_keys) == 0:
    print("🎉 ALL SYSTEMS GO! You're ready to build StockSage.AI!")
else:
    print("\n📝 TO DO:")
    print("  1. Get API keys from the URLs shown above")
    print("  2. Add them to your .env file")
    print("  3. Rerun this notebook")
    
print("\n🔗 Quick API registration links:")
print("  • FRED: https://fred.stlouisfed.org/docs/api/")
print("  • NewsAPI: https://newsapi.org/")
print("  • OpenAI: https://platform.openai.com/api-keys")

print(f"\n✨ Next: Start with data collection scripts in src/data/!")


🎯 STOCKSAGE.AI - SYSTEM STATUS CHECK

✅ WORKING SYSTEMS:
  📊 Core Libraries (pandas, plotly)
  📈 Market Data (yfinance)
  🏛️ Economic Data (FRED)
  📰 News Data (NewsAPI)
  🤖 OpenAI API

⚠️  NEEDS SETUP:

🚀 STATUS: 5/5 systems ready
🎉 ALL SYSTEMS GO! You're ready to build StockSage.AI!

🔗 Quick API registration links:
  • FRED: https://fred.stlouisfed.org/docs/api/
  • NewsAPI: https://newsapi.org/
  • OpenAI: https://platform.openai.com/api-keys

✨ Next: Start with data collection scripts in src/data/!
